In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'2.1.0'

In [2]:
x_data = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

W1 = tf.Variable(tf.random.normal([2, 1]), dtype=tf.float32)
b1 = tf.Variable(tf.random.normal([1, ]), dtype=tf.float32)
                 
W2 = tf.Variable(tf.random.normal([2, 1]), dtype=tf.float32)
b2 = tf.Variable(tf.random.normal([1, ]), dtype=tf.float32)

W3 = tf.Variable(tf.random.normal([2, 1]), dtype=tf.float32)
b3 = tf.Variable(tf.random.normal([1, ]), dtype=tf.float32)

variables = [W1, W2, W3, b1, b2, b3]

In [3]:
def preprocess_fn(features, labels):
    features = tf.cast(features, dtype=tf.float32)
    labels = tf.cast(labels, dtype=tf.float32)
    return features, labels

In [4]:
def hypothesis_fn(features):
    layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
    layer2 = tf.sigmoid(tf.matmul(features, W2) + b2)
    layer3 = tf.concat([layer1, layer2], -1)
    layer3 = tf.reshape(layer3, [-1, 2])
    return tf.sigmoid(tf.matmul(layer3, W3) + b3)

In [5]:
def loss_fn(hypothesis, labels):
    return -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))

In [6]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [11]:
def grad_fn(features, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(hypothesis_fn(features), labels)
    return tape.gradient(loss, variables)

In [12]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
EPOCHS = 50000

for step in range(EPOCHS):
    for features, labels in dataset:
        features, labels  = preprocess_fn(features, labels)
        grads = grad_fn(features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
        if step % 5000 == 0:
            print("Iter: {:4d} | Loss: {:.4f}".format(step, loss_fn(hypothesis_fn(features), labels)))

Iter:    0 | Loss: 0.7068
Iter: 5000 | Loss: 0.6508
Iter: 10000 | Loss: 0.5774
Iter: 15000 | Loss: 0.5054
Iter: 20000 | Loss: 0.4429
Iter: 25000 | Loss: 0.3199
Iter: 30000 | Loss: 0.1906
Iter: 35000 | Loss: 0.1187
Iter: 40000 | Loss: 0.0817
Iter: 45000 | Loss: 0.0609


NameError: name 'preprocess_data' is not defined

In [14]:
x_data, y_data = preprocess_fn(x_data, y_data)
test_acc = accuracy_fn(hypothesis_fn(x_data), y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 1.0000
